In [2]:
%run mathutil.ipynb
np.random.seed(1234)
def randomize(): np.random.seed(time.time())

## Model 클래스 선언

In [3]:
class Model(object):
    def __init__(self, name, dataset):
        self.name = name
        self.dataset = dataset
        self.is_training = False #학습 중 여부를 나타내는 플래그
        if not hasattr(self, 'rand_std'):
            self.rand_std = 0.030
    
    def __str__(self): #출력문으로 객체를 출력할 때의 출력 문자열 생성방법 정의
        return '{}/{}'.format(self.name, self.dataset)

    def exec_all(self, epoch_count = 10, batch_size = 10, learning_rate = 0.001, report = 0, show_cnt = 3):
        #전체 과정을 실행시키는 메인 함수
        self.train(epoch_count, batch_size, learning_rate, report)
        self.test()
        if show_cnt > 0: self.visualize(show_cnt)

## MlpModel 클래스 선언

In [4]:
class MlpModel(Model): #Model의 파생 클래스로 선언
    #객체 초기화 메서드 __init__을 제외한 나머지 메서드들을 클래스 선언 밖에서 따로 함수를 정의해 메서드로 등록하는 방식
    def __init__(self, name, dataset, hconfigs):
        super(MlpModel, self).__init__(name, dataset) #super 명령을 이용해 기반 클래스인 Model클래스를 찾아 그 객체 초기화 함수를 호출하여 name, dataset 저장
        self.init_parameters(hconfigs) #신경망이 이용할 파라미터 준비

다층 퍼셉트론의 은닉 계층 구성은 init_parameters 메서드에 전달되는 hconfig 인숫값에 따라 결정

## 파라미터 생성 메서드의 정의

In [5]:
def mlp_init_parameters(self, hconfigs):
    self.hconfigs = hconfigs
    self.pm_hiddens = []

    prev_shape = self.dataset.input_shape #입출력 벡터의 크기 등의 정보는 전역변수가 아닌 dataset 객체의 속성값으로 얻음

    for hconfig in hconfigs:
        pm_hidden, prev_shape = self.alloc_layer_param(prev_shape, hconfig)
        self.pm_hiddens.append(pm_hidden) #생성된 파라미터들을 전역 변수에 저장하는 대신 객체변수로 저장

    output_cnt = int(np.prod(self.dataset.output_shape)) #파라미터 생성에 반영되는 벡터 크기를 np.prod 함수를 이용해 변환
    #확장 과정에서 입출력 벡터 크기나 은닉 벡터 크기가 자연수가 아닌 다차원 크기를 나타내는 리스트나 튜플의 형채로도 표현되기 때문
    self.pm_output, _ = self.alloc_layer_param(prev_shape, output_cnt)

def mlp_alloc_layer_param(self, input_cnt, output_cnt):
    input_cnt = np.prod(input_cnt)
    output_cnt = hconfig

    weight, bias = self.alloc_param_pair([input_cnt, output_cnt])
    return {'w':weight, 'b':bias}, output_cnt

def mlp_alloc_param_pair(self, shape):
    weight = np.random.normal(0, self.rand_std, shape)
    bias = np.zeros([shape[-1]])
    return weight, bias

MlpModel.init_parameters = mlp_init_parameters
MlpModel.alloc_layer_param = mlp_alloc_layer_param
MlpModel.alloc_param_pair = mlp_alloc_param_pair

init_parameters 메서드는 직접 모든 일을 처리하지 않고 계층 하나의 파라미터를 생성할 때마다 alloc_layer_param 메서드를 부름

alloc_layer_param 역시 alloc_param_pair메서드를 불러 가중치와 편향 파라미터 생성

## 학습 메서드

In [ ]:
def mlp_model_train(self, epoch_count = 10, batch_size = 10, learning_rate = 0.001, report= 0):
    self.learning_rate = learning_rate

    batch_count = int(self.dataset.train_count / batch_size)
    time1 = time2 = int(time.time())
    if report != 0:
        print('Model {} train started:'.format(self.name))

    for epoch in epoch_count:
        costs = []
        accs = []
        self.dataset.shuffle_train_data(batch_size * batch_count)

        for n in range(batch_count):
            trX, trY = self.dataset.get_train_data(batch_size, n)
            cost, acc = self.train_step(trX, trY)
            costs.append(cost)
            accs.append(acc)

        if report > 0 and (epoch + 1) % report == 0:
            vaX, vaY = self.dataset.get_validate_data(100)
            acc = self.eval_accuracy(vaX, vaY)
            time3 = int(time.time())
            tm1, tm2 = time3-time2, time3-time1
            self.dataset.train_prt_result(epoch+1, costs, accs, acc, tm1, tm2)
            time2 = time3

    tm_total = int(time.time() - time1)
    print('Model () train ended in () secs:'.format(self.name, tm_total))

MlpModel.train = mlp_model_train
